In [1]:
!pip install torch-xla -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in links: https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 5.5 MB/s eta 0:00:00


In [2]:
!pip install torch transformers pandas scikit-learn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
sinhala_fake1 = pd.read_csv('/content/drive/MyDrive/RP/sinhala_fake.csv')
sinhala_fake2 = pd.read_excel('/content/drive/MyDrive/RP/sinhala_fake_2.xlsx')
sinhala_real = pd.read_excel('/content/drive/MyDrive/RP/sinhala_real.xlsx')

In [6]:
sinhala_fake = pd.concat([sinhala_fake1.drop('title', axis=1), sinhala_fake2.rename(columns={'title_sinhala': 'text'})], axis=0).reset_index(drop=True)

In [7]:
sinhala_real.drop(['id','news_url','title'], axis=1, inplace=True)
sinhala_real.rename(columns={'title_sinhala': 'text'}, inplace=True)

In [8]:
sinhala_fake['label'] = 1
sinhala_real['label'] = 0

In [9]:
sinhala_news_df = pd.concat([sinhala_fake, sinhala_real], axis=0).reset_index(drop=True)

In [10]:
sinhala_news_df.head(5)

,text,label
0,Miley Cyrus සහ Liam Hemsworth රහසේ විවාහ වුණාද?,1
1,පැරිස් ජැක්සන් සහ කාරා ඩෙලිවින්ග්න් ගැලපෙන ඇඳු...,1
2,ඩොනල්ඩ් ට්‍රම්ප්ට විරෝධය පළ කරමින් ප්‍රසිද්ධ ප...,1
3,සින්ඩි ක්‍රෝෆර්ඩ්ගේ දියණිය Kaia Gerber හැරී ස්...,1
4,2018 ඔස්කාර් නාමයෝජනා සම්පූර්ණ ලැයිස්තුව - විව...,1


In [11]:
# dataset is in a DataFrame named 'train_data' and the label column is named 'label'
label_counts = sinhala_news_df['label'].value_counts()
print("Class distribution in training data:")
print(label_counts)

Class distribution in training data:
label
0    5999
1    5331
Name: count, dtype: int64


In [14]:
import pandas as pd
import re

# Define the cleaning function
def clean_text(text):
    if not isinstance(text, str):  # Check if the value is not a string
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^අ-෴a-zA-Z\s]", "", text)  # Keep Sinhala and English letters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Apply the cleaning function safely
sinhala_news_df['text'] = sinhala_news_df['text'].fillna("").apply(clean_text)

In [13]:
sinhala_news_df.head(5)

,text,label
0,Miley Cyrus සහ Liam Hemsworth රහසේ විවාහ වුණාද,1
1,පැරිස් ජැක්සන් සහ කාරා ඩෙලිවින්ග්න් ගැලපෙන ඇඳු...,1
2,ඩොනල්ඩ් ට්රම්ප්ට විරෝධය පළ කරමින් ප්රසිද්ධ පුද...,1
3,සින්ඩි ක්රෝෆර්ඩ්ගේ දියණිය Kaia Gerber හැරී ස්ට...,1
4,ඔස්කාර් නාමයෝජනා සම්පූර්ණ ලැයිස්තුව විවිධත්වය,1


In [15]:
sinhala_news_df.shape

(11330, 2)

In [16]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load SinBERT-large
tokenizer = AutoTokenizer.from_pretrained("NLPC-UOM/SinBERT-large")
model = AutoModelForMaskedLM.from_pretrained("NLPC-UOM/SinBERT-large")

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "NLPC-UOM/SinBERT-large",
    num_labels=2  # Binary classification (FAKE, REAL)
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at NLPC-UOM/SinBERT-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
